<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<h4>محدودیت‌های امنیتی سیستم عامل</h4>
<p>تمامی برنامه‌ها تحت محدودیت‌های امنیتی سیستم عامل و بر اساس سطح دسترسی کاربر اجرا می‌شوند. این محدودیت‌ها می‌تواند بر دسترسی به فایل‌ها، رجیستری ویندوز و دیگر قابلیت‌ها تاثیر بگذارد.</p>

<h4>ویندوز و کنترل حساب کاربری (User Account Control - UAC)</h4>
<p>در ویندوز، ویژگی‌ای به نام UAC وجود دارد که به کاربران مدیریتی دو توکن (token) یا "کلاه" (hat) می‌دهد:</p>
<ul><li><strong>کلاه مدیریتی</strong>: برای دسترسی‌های کامل به سیستم.</li><li><strong>کلاه کاربر معمولی</strong>: با دسترسی‌های محدود.</li></ul>

<p>به صورت پیش‌فرض، برنامه‌ها با کلاه کاربر معمولی اجرا می‌شوند، مگر اینکه برنامه درخواست افزایش سطح دسترسی کند و کاربر آن را تایید کند.</p>

<h4>اجرای برنامه با دسترسی محدود</h4>
<p>به طور پیش‌فرض، برنامه شما با دسترسی‌های محدود اجرا می‌شود. برای این منظور، شما می‌توانید یکی از دو رویکرد زیر را انتخاب کنید:</p>
<ol><li><strong>نوشتن برنامه به گونه‌ای که بدون دسترسی مدیریتی اجرا شود</strong>:<ul><li>این روش ایمن‌تر و راحت‌تر برای کاربر است و در بسیاری از موارد نیز ساده‌تر است.</li></ul></li><li><strong>درخواست افزایش سطح دسترسی در ویندوز (Windows only)</strong>:<ul><li>درخواست افزایش سطح دسترسی در مانفیست برنامه یا تشخیص عدم دسترسی مورد نیاز و اعلام به کاربر برای اجرای مجدد برنامه به صورت مدیریتی.</li></ul></li></ol>
</div>

In [ ]:
using System.Runtime.InteropServices;
using System.Security.Principal;

[DllImport("libc")]
public static extern uint getuid();
/*
تابع getuid از کتابخانه libc (کتابخانه استاندارد C)
 برای سیستم‌های یونیکس/لینوکس وارد می‌شود
تابع getuid شناسه کاربری موثر (effective user ID)
 فرآیند جاری را برمی‌گرداند.
  در سیستم‌های یونیکس/لینوکس، شناسه کاربری ریشه (root) برابر با 0 است. 
 */

static bool IsRunningAsAdmin()
{
    //for Windows
    if (RuntimeInformation.IsOSPlatform (OSPlatform.Windows))
    {
        using var identity = WindowsIdentity.GetCurrent();
        var principal = new WindowsPrincipal (identity);
        return principal.IsInRole (WindowsBuiltInRole.Administrator);
    }

    //for unix or linux
    return getuid() == 0;
}

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">

<h4>محدودیت‌های حساب کاربری استاندارد</h4>
<p>در یک حساب کاربری استاندارد، موارد زیر امکان‌پذیر نیست:</p>
<ol><li><p><strong>نوشتن در دایرکتوری‌های خاص:</strong></p><ul><li><strong>پوشه سیستم عامل (OS folder) و زیرپوشه‌ها</strong>:<ul><li>در ویندوز: <code>\Windows</code></li><li>در یونیکس/لینوکس: <code>/bin</code>, <code>/sbin</code> و زیرپوشه‌های آنها</li></ul></li><li><strong>پوشه برنامه‌ها (Program Files folder) و زیرپوشه‌ها</strong>:<ul><li>در ویندوز: <code>\Program Files</code></li><li>در یونیکس/لینوکس: <code>/usr/bin</code>, <code>/opt</code> و زیرپوشه‌های آنها</li></ul></li><li><strong>ریشه درایو سیستم عامل (Root of the OS drive)</strong>:<ul><li>در ویندوز: <code>C:\</code></li><li>در یونیکس/لینوکس: <code>/</code></li></ul></li></ul></li><li><p><strong>نوشتن در شاخه HKEY_LOCAL_MACHINE رجیستری ویندوز:</strong></p><ul><li>این شاخه شامل تنظیمات سیستم گسترده (system-wide) است که نیاز به دسترسی‌های مدیریتی دارد.</li></ul></li><li><p><strong>خواندن داده‌های مانیتورینگ عملکرد (Performance Monitoring) (WMI) در ویندوز:</strong></p><ul><li>دسترسی به داده‌های WMI معمولاً محدود به کاربران مدیریتی است.</li></ul></li></ol>

<h4>محدودیت‌های دسترسی به فایل‌ها و منابع دیگر کاربران</h4>
<p>به عنوان یک کاربر معمولی ویندوز (یا حتی به عنوان یک مدیر سیستم)، ممکن است دسترسی به فایل‌ها یا منابع متعلق به کاربران دیگر محدود باشد. ویندوز از یک سیستم لیست‌های کنترل دسترسی (Access Control Lists - ACLs) استفاده می‌کند تا از این منابع محافظت کند. شما می‌توانید حقوق خود را در ACLها با استفاده از انواع موجود در فضای نام <code>System.Security.AccessControl</code> پرس و جو و اعلام کنید.</p>
</div>

In [ ]:
//The recommended location for user documents is SpecialFolder.MyDocuments:

string docsFolder = Environment.GetFolderPath (Environment.SpecialFolder.MyDocuments);
string path = Path.Combine (docsFolder, "test.txt");

The recommended location for **configuration files** that a user might need to `modify` ***outside of your application*** is `SpecialFolder.ApplicationData` (***current user only***) or `SpecialFolder.CommonApplicationData` (***all users***).

### Administrative Elevation and Virtualization

With an `application manifest`, you can **request** that **Windows prompt** the user for `administrative elevation` whenever running your program (**Linux ignores this
request**)

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<assembly manifestVersion="1.0" xmlns="urn:schemas-microsoft-com:asm.v1">
    <trustInfo xmlns="urn:schemas-microsoft-com:asm.v2">
        <security>
            <requestedPrivileges>
                <requestedExecutionLevel level="requireAdministrator" />
            </requestedPrivileges>
        </security>
    </trustInfo>
</assembly>